In [1]:
# Quick setup
import sys
import pandas as pd
import numpy as np
from src.utils.notebook_setup import setup_notebook_environment
dbs, logger = await setup_notebook_environment()

# Now ready to work
logger.info("Starting data ingestion...")


2025-07-25 18:27:48,998 - api.services.database_service - INFO - Connected to MongoDB database: healthcare
2025-07-25 18:27:49,003 - src.utils.notebook_setup - INFO - Database connected: True
2025-07-25 18:27:49,004 - src.utils.notebook_setup - INFO - Database collections: ['heart_disease_silver', 'heart_disease_bronze', 'heart_disease_gold']
2025-07-25 18:27:49,005 - src.utils.notebook_setup - INFO - Database collections count: 3
2025-07-25 18:27:49,006 - src.utils.notebook_setup - INFO - Starting data ingestion...


In [2]:
# Load CSV -> Insert into Bronze layer

df = pd.read_csv("../data/raw/heart_disease_uci.csv")

logger.debug(df.head())
logger.debug(df.shape)
logger.debug(df.columns)
logger.debug(df.dtypes)
logger.debug(df.isnull().sum())

# Insert into Bronze layer
data_dict = df.to_dict("records")
success = await dbs.insert_bronze_data(data_dict)

bronze_data = await dbs.get_bronze_data()
bronze_data_df = pd.DataFrame(bronze_data)
logger.info(bronze_data_df.head())

logger.info("=" * 60)
logger.info("BRONZE LAYER DATA SUMMARY")
logger.info("=" * 60)
logger.info(f"Final dataset shape: {bronze_data_df.shape}")
# Show final columns
logger.info(f"Final columns: {list(bronze_data_df.columns)}")

# Save to Bronze collection in MongoDB
logger.info("Saving raw unprocessed data to Bronze layer...")


2025-07-25 18:27:49,069 - api.services.database_service - INFO - Inserted 920 records into Bronze layer
2025-07-25 18:27:49,117 - src.utils.notebook_setup - INFO -                         _id  id  age     sex    dataset               cp  \
0  6883eca61a6b29690a5b3e9f   1   63    Male  Cleveland   typical angina   
1  6883eca61a6b29690a5b3ea0   2   67    Male  Cleveland     asymptomatic   
2  6883eca61a6b29690a5b3ea1   3   67    Male  Cleveland     asymptomatic   
3  6883eca61a6b29690a5b3ea2   4   37    Male  Cleveland      non-anginal   
4  6883eca61a6b29690a5b3ea3   5   41  Female  Cleveland  atypical angina   

   trestbps   chol    fbs         restecg  thalch  exang  oldpeak  \
0     145.0  233.0   True  lv hypertrophy   150.0  False      2.3   
1     160.0  286.0  False  lv hypertrophy   108.0   True      1.5   
2     120.0  229.0  False  lv hypertrophy   129.0   True      2.6   
3     130.0  250.0  False          normal   187.0  False      3.5   
4     130.0  204.0  False  lv hype